In [4]:
import pandas as pd

In [5]:
class feature_engine:
    
    pd = __import__('pandas')
    datetime = __import__('datetime')
    mcal = __import__('pandas_market_calendars')
    np = __import__('numpy')
    
    def __init__(self):
        print('feature_engine V.0.1 \nImported pandas,datetime,pandas_market_calendars,numpy packages')
        
    def calc_price_change(self,tweet_time,financial_data,time_col_name,price_col_name,interval,amount,method='both',
                          logdiff=True):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
        #default baseline is set for five minutes before tweet
        tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
        tweet_price=financial_data[financial_data[time_col_name]==tweet_time_5min_before][price_col_name]
        if len(tweet_price)==0:
            return float('NaN')
        tweet_price.reset_index(inplace=True,drop=True)

        def calculation(financial_data,time_col_name,new_time,price_col_name,tweet_price,direction,logdiff):
            
            other_price=financial_data[financial_data[time_col_name]==new_time][price_col_name]
            other_price.reset_index(inplace=True,drop=True)
            if len(other_price)==0:
                return float('NaN')
            elif logdiff:
                if direction=='forward':
                    return (self.np.log(other_price.iloc[0]) - self.np.log(tweet_price.iloc[0]))
                elif direction=='backward':
                    return (self.np.log(tweet_price.iloc[0]) - self.np.log(other_price.iloc[0]))
            else:
                if direction=='forward':
                    return ((other_price.iloc[0]/tweet_price.iloc[0])-1)
                elif direction=='backward':
                    return ((tweet_price.iloc[0]/other_price.iloc[0])-1)

        if (method=='both') or (method=='forward'):  
            if interval=='minutes':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
            elif interval=='hours':
                time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
            elif interval=='days':
                time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
            pct_change_forward=calculation(financial_data,time_col_name,time_forward,price_col_name,tweet_price,'forward',
                                          logdiff)

        if (method=='both') or (method=='backward'): 
            if interval=='minutes':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)   
            elif interval=='hours':
                time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount) 
            elif interval=='days':
                time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
            pct_change_backward=calculation(financial_data,time_col_name,time_backward,price_col_name,tweet_price,'backward'
                                           ,logdiff)
        
        if method == 'both':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward,
                          f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='backward':
            return self.pd.DataFrame({f'{amount}_{interval}_backward_pct_change':pct_change_backward},index=[0])
        elif method=='forward':
            return self.pd.DataFrame({f'{amount}_{interval}_forward_pct_change':pct_change_forward},index=[0])
        
    def create_pricechg_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,price_col_name,
                       interval_amount_dict,method='both',logdiff=True):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calc_price_change(twitter_data[tweet_time_col_name].loc[i],financial_data,fin_time_col_name,
                                                   price_col_name,interval,amount,method,logdiff)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns
    
    def mean_encoding_tocolumn(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        grouped=features_df.groupby([categorical_column]).mean()
        features_df=features_df.merge(grouped,on=categorical_column)
        return features_df[f'{target_column}_y']
    
    def mean_encoding_todict(self,features_df,categorical_column,target_column):
        features_df=features_df[[categorical_column,target_column]]
        return features_df.groupby([categorical_column]).mean()
    
    def calculate_sum_volume_diff(self,tweet_time,financial_data,time_col_name,volume_col_name,interval,amount):
        #remove seconds to match financial data format
        tweet_time=tweet_time.replace(second=0)
#         default baseline is set for five minutes before tweet?
#       tweet_time_5min_before=tweet_time - self.datetime.timedelta(0,0,0,0,5)
        
        def calculation(financial_data,time_col_name,time_forward,time_backward,volume_col_name,tweet_time):
            forward_volume_sum=sum(financial_data.loc[(financial_data[time_col_name]<=new_time) & 
                                        (financial_data[time_col_name]>=tweet_time)][volume_col_name])
            backward_volume_sum=sum(financial_data.loc[(financial_data[time_col_name]<=tweet_time) & 
                                        (financial_data[time_col_name]>=time_backward)][volume_col_name])
            return self.np.log(forward_volume_sum)-self.np.log(backward_volume_sum)

        if interval=='minutes':
            time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,amount)
            time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,amount)
        elif interval=='hours':
            time_forward=tweet_time + self.datetime.timedelta(0,0,0,0,0,amount)
            time_backward=tweet_time - self.datetime.timedelta(0,0,0,0,0,amount)
        elif interval=='days':
            time_forward=tweet_time + self.datetime.timedelta(amount,0,0,0,0)
            time_backward=tweet_time - self.datetime.timedelta(amount,0,0,0,0)
        vol_sum_logdiff=calculation(financial_data,time_col_name,time_forward,time_backward,volume_col_name,tweet_time)

        return self.pd.DataFrame({f'{amount}_{interval}_forward_vol_sum':vol_sum_forward},index=[0])
        
    def create_volumesum_columns(self,twitter_data,tweet_time_col_name,financial_data,fin_time_col_name,volume_col_name,
                       interval_amount_dict):
        from tqdm import tqdm
        new_columns=self.pd.DataFrame()
        for i, row in tqdm(twitter_data.iterrows()):
            temp_df=self.pd.DataFrame()
            for interval, amount in interval_amount_dict:
                new_df=self.calculate_sum_volume_diff(twitter_data[tweet_time_col_name].loc[i],financial_data,fin_time_col_name,
                                                   volume_col_name,interval,amount)
                if not isinstance(new_df,float):
                    temp_df=self.pd.concat([temp_df,new_df],axis=1)
            for item in temp_df.columns:
                new_columns.at[i,item] = temp_df.loc[0,item]
        return new_columns
    
    def fill_missing_fin_data(self,ticker_col_name,stock_exchg_name,start_date,end_date,frequency,fin_time_colname,
                          financial_data,volume_colname,price_colname):
        ticker_name=financial_data.iloc[0][ticker_col_name]
        stock_exchg = self.mcal.get_calendar(stock_exchg_name, open_time=self.datetime.time(5, 30), 
                                             close_time=self.datetime.time(12, 0))
        daterange = stock_exchg.schedule(start_date, end_date)
        dates=self.pd.DataFrame(self.mcal.date_range(daterange, frequency).tz_convert(None),columns=[fin_time_colname])

#         financial_data[fin_time_colname] = self.pd.to_datetime(financial_data[fin_time_colname], utc = True)

        financial_data=dates.merge(financial_data,on=fin_time_colname,how='left')
        financial_data[ticker_col_name]=ticker_name

        financial_data['SYM_SUFFIX'].fillna(0,inplace=True)

        financial_data[volume_colname].fillna(0,inplace=True)
        financial_data[price_colname].interpolate(inplace=True)
        return financial_data
    
    def token_matrix(self,text_column,financial_topic_words):
        
        def check_for_word(token_vector,word):
            if word in token_vector:
                return 1
            else:
                return 0
            
        new_df=self.pd.DataFrame()
        for topic_word in fin_topic_words:
            new_df[topic_word]=text_column.apply(lambda x:check_for_word(x.lower().split(),topic_word))
            
        return new_df
    
    def diff_from_meanlog(self,df,date_colname,numeric_colname):
        #This function returns the difference of logs between the original value and that month's average
        ins_df=df[[date_colname,numeric_colname]].copy().set_index(date_colname,drop=True).astype(int).resample("M").median()
        ins_df[numeric_colname]=self.np.log(ins_df[numeric_colname])
        ins_df['month']=ins_df.index.map(str).str[:7]
        df['month']=df[date_colname].map(str).str[:7]
        averages=df[[date_colname,'month']].merge(ins_df,on='month',how='left')[numeric_colname]
        diff=self.np.log(df[numeric_colname])-averages
        return diff

In [6]:
financial_data=pd.read_csv('04052009-18122019-SPYDIA-minutedata.csv',parse_dates=['DATETIME'])
twitter_data=pd.read_csv('trump_tweets_reducted_after_nlp.csv',index_col=0,parse_dates=['created_at_utc'])

C:\Users\natan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,6,8,11,12,13,15,16,17,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
sp500_data=financial_data[financial_data['SYM_ROOT']=='SPY']
dow_data=financial_data[financial_data['SYM_ROOT']=='DIA']

In [8]:
feat_eng = feature_engine()

feature_engine V.0.1 
Imported pandas,datetime,pandas_market_calendars,numpy packages


In [9]:
twitter_data['retweet_count']=feat_eng.diff_from_meanlog(twitter_data,'created_at_utc','retweet_count')
twitter_data['favorite_count']=feat_eng.diff_from_meanlog(twitter_data,'created_at_utc','favorite_count')

C:\Users\natan\Anaconda3\lib\site-packages\pandas\core\series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
sp500_data=feat_eng.fill_missing_fin_data('SYM_ROOT','NYSE','2009-05-04','2019-12-18','1min','DATETIME',
                          sp500_data,'SIZE','PRICE')

In [11]:
dow_data=feat_eng.fill_missing_fin_data('SYM_ROOT','NYSE','2009-05-04','2019-12-18','1min','DATETIME',
                          dow_data,'SIZE','PRICE')

In [7]:
tweets_finance=feat_eng.create_pricechg_columns(twitter_data,'created_at_utc',dow_data,'DATETIME','PRICE',
                       [('minutes',1),('minutes',5),('minutes',10),('minutes',15),('minutes',30),('hours',1),
                        ('hours',3),('hours',6),('days',1),('days',5),('days',10),('days',30)],method='forward')

25702it [1:29:59,  3.62it/s]


In [8]:
tweets_finance_volume=feat_eng.create_volumesum_columns(twitter_data,'created_at_utc',dow_data,'DATETIME','SIZE',
                       [('minutes',1),('minutes',5),('minutes',10),('minutes',15),('minutes',30),('hours',1),
                        ('hours',3),('hours',6),('days',1),('days',5),('days',10),('days',30)],method='forward')

25702it [6:52:35,  1.32it/s]


In [9]:
twitter_data_with_finance=twitter_data.join(tweets_finance)

In [10]:
twitter_data_with_finance=twitter_data_with_finance.join(tweets_finance_volume)

In [31]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance.copy()

In [32]:
twitter_data_with_finance_for_analysis.drop(['coordinates','extended_entities','geo','hashtags','in_reply_to_screen_name',
                                             'in_reply_to_status_id','in_reply_to_user_id','is_quote_status','is_retweet',
                                             'lang','place','possibly_sensitive','quoted_status','quoted_status_id','id',
                                             'retweeted_status','retrieved_utc'],1,inplace=True)

In [33]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance_for_analysis.join(pd.get_dummies(twitter_data_with_finance_for_analysis['source']))

In [34]:
twitter_data_with_finance_for_analysis.drop(['source','truncated','user_mentions'],1,inplace=True)

In [35]:
fin_topic_words=['deal','deals','china','chinese','nafta','market','markets','stock','stocks','trade','trades','trading','tax',
             'taxes','taxation','rate','rates','unemployment','jobs','manafacture','manafacturers','dollar','dollars','$',
             'billion','billions','consumer','consumers','reserve','gdp','growth','bank','banks','debt','factories',
             'revenue','spending','deficit','economy','economies','economist','economic','economists','money',
             'tariff','tariffs','companies','inflation','refinance','finance','financial','currency','depreciating',
             'depreceate','fed','business','businesses','bankruptcy','price','prices','products','opec','investment',
             'savings','cents','cent','corporate','corporates','deficits','dow','nasdaq','labor','purchase','500',
             'industry','industries']

In [36]:
financial_words_matrix=feat_eng.token_matrix(twitter_data_with_finance_for_analysis['text'],fin_topic_words)

In [37]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance_for_analysis.join(financial_words_matrix)

In [38]:
twitter_data_with_finance_for_analysis['trump_president']=twitter_data_with_finance_for_analysis['created_at_utc']>='2016-11-08 22:00:00'

In [19]:
twitter_data_with_finance_for_analysis.drop('text',1,inplace=True)

In [20]:
twitter_data_with_finance_for_analysis=twitter_data_with_finance_for_analysis.join(pd.get_dummies(twitter_data_with_finance_for_analysis['lda_topic']))

In [21]:
twitter_data_with_finance_for_analysis.drop('lda_topic',1,inplace=True)

In [22]:
twitter_data_with_finance_for_analysis.to_csv('twitter_data_with_dowjones_for_analysis.csv')

In [48]:
for x in twitter_data_with_finance_for_analysis[twitter_data_with_finance_for_analysis['currency']==1]['text']:
    print(x)

China is stealing our jobs. We need to demand China stop manipulating its currency and end its rampant corporate espionage.
China is threatening Washington over the currency bill.  We should pass it immediately.
Stop "The China Curse" -- Pass the Chinese Currency Bill!
@Johnboehner: Stop "The China Curse" --- Pass the China Currency Bill!
When will Washington stand up to China. China is manipulating its currency and stealing our jobs. Washington should move on legislation.
Getting China to stop playing its currency charades can begin whenever we elect a president ready to take decisive action. #TimeToGetTough
“China presents three big threats to the United States in its outrageous currency manipulation its systematic (cont) http://t.co/6hW0AcJE
Why is @MittRomney the only guy who talks about getting tough with China and their currency manipulation?
The military threat from China is gigantic--and it's no surprise that the Communist Chinese government lies (cont) http://t.co/n6AbRrUY,03-